In [1]:
import os.path
import warnings
import tensorflow as tf
import glob
import project_tests as ptests
import helper

In [2]:
# Global variables for paths
data_dir = './data'
runs_dir = './runs'
training_dir = data_dir + '/data_road/training'
training_size = len(glob.glob(training_dir + '/calib/*.*'))
vgg_dir = data_dir + '/vgg'

In [3]:
# Check Tensorflow Version
from distutils.version import LooseVersion
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Assertion Failed. Tensorflow > 1.0 current version is {}'.format(tf.__version__)

# Print version if assertion is successful
print('Assertion Successful: TF Version: {}'.format(tf.__version__))

Assertion Successful: TF Version: 1.2.1


In [4]:
# Check GPU availability
from tensorflow import test as tft
if not tft.gpu_device_name():
  warnings.warn('GPU not found... Please reconsider working with a GPU for training')
else:
  print('Default GPU : {}'.format(tft.gpu_device_name()))

Default GPU : /gpu:0


In [5]:
# Training constants
num_classes_ = 2
img_shape_ = (160, 576)

epochs_ = 20
batch_size_ = 1

learning_rate_ = 0.0001
dropout_ = 0.75

In [6]:
# Place_holders (_ph)
label_ph = tf.placeholder(tf.float32, [None, 
                                       img_shape_[0],
                                       img_shape_[1], 
                                       num_classes_])

# Learning Rate
learning_rate_ph = tf.placeholder(tf.float32)

# keep_prob
keep_prob_ph = tf.placeholder(tf.float32)

In [7]:
# Initialize training losses to null
all_training_losses = [] 

In [8]:
def load_vgg(sess, vgg_dir):
    """
    Load Pretrained VGG Model
    @param sess:  Tf Session
    @param vgg_dir: Directory containing vgg "variables/" and "saved_model.pb"
    return: VGG Tensor Tuple(image_input, keep_prob, layer3, layer4, layer7)
    """
    # Load Model with Weights from vgg directory
    model = tf.saved_model.loader.load(sess, ['vgg16'], vgg_dir)

    # Setup tensors to get from graph ( vgg after loading)
    graph = tf.get_default_graph()

    # get image input
    image_input = graph.get_tensor_by_name('image_input:0')

    # get keep probability
    keep_prob_ph = graph.get_tensor_by_name('keep_prob:0')

    # Get layer outputs
    layer_3 = graph.get_tensor_by_name('layer3_out:0')
    layer_4 = graph.get_tensor_by_name('layer4_out:0')
    layer_7 = graph.get_tensor_by_name('layer7_out:0')

    # return as 5D list
    return image_input, keep_prob_ph, layer_3, layer_4, layer_7

In [9]:
def conv_1x1(layer, layer_name):
  """ convolve layer by (1x1) to preserve spatial information """
  return tf.layers.conv2d(inputs = layer,
                          filters =  num_classes_,
                          kernel_size = (1, 1),
                          strides = (1, 1),
                          name = layer_name)

In [10]:
def deconvolve(layer, k, s, layer_name):
  """ Transpose Convolve/ deconvolve a layer with arguments as params """
  return tf.layers.conv2d_transpose(inputs = layer,
                                    filters = num_classes_,
                                    kernel_size = (k, k),
                                    strides = (s, s),
                                    padding = 'same',
                                    name = layer_name)

In [11]:
def layers(vgg_layer_3_out, vgg_layer_4_out, vgg_layer_7_out, num_classes = num_classes_):
    """
    # Create layers for the FCN.
    vgg_layer_n_out: TF Tensor for VGG Layer n output
    num_classes: Number of classes to classify
    return: The Tensor for the last layer of output
    """


    # Apply a 1x1 convolution to all argument layers
    layer_3x = conv_1x1(layer = vgg_layer_3_out, layer_name = "layer3conv1x1")
    layer_4x = conv_1x1(layer = vgg_layer_4_out, layer_name = "layer4conv1x1")
    layer_7x = conv_1x1(layer = vgg_layer_7_out, layer_name = "layer7conv1x1")

    # Add decoder layers to the network with skip connections
    # Deconvolve
    decoder_layer_1 = deconvolve(layer = layer_7x, k = 4, s = 2, layer_name = "decoderlayer1")
    
    # Sum (skip connection)
    decoder_layer_2 = tf.add(decoder_layer_1, layer_4x, name = "decoderlayer2")
    
    # Deconvolve
    decoder_layer_3 = deconvolve(layer = decoder_layer_2, k = 4, s = 2, layer_name = "decoderlayer3")

    # Sum (skip connection)
    decoder_layer_4 = tf.add(decoder_layer_3, layer_3x, name = "decoderlayer4")
    
    # Deconvolve
    decoderlayer_output = deconvolve(layer = decoder_layer_4, k = 16, s = 8, layer_name = "decoderlayer_output")

    return decoderlayer_output

In [12]:
def layers_verbose(vgg_layer_3_out, vgg_layer_4_out, vgg_layer7_out, num_classes = num_classes_):

    """
    # Create layers for the FCN.
    vgg_layer_n_out: TF Tensor for VGG Layer n output
    num_classes: Number of classes to classify
    return: List of tensors for all layers
    """
    # Apply a 1x1 convolution to encoder layers
    layer3x = conv_1x1(layer = vgg_layer_3_out, layer_name = "layer3conv1x1")
    layer4x = conv_1x1(layer = vgg_layer_4_out, layer_name = "layer4conv1x1")
    layer7x = conv_1x1(layer = vgg_layer7_out, layer_name = "layer7conv1x1")

    # Add decoder layers to the network with skip connections
    # Deconvolve
    decoder_layer_1 = deconvolve(layer = layer_7x, k = 4, s = 2, layer_name = "decoderlayer1")
    
    # Sum (skip connection)
    decoder_layer_2 = tf.add(decoder_layer_1, layer_4x, name = "decoderlayer2")
    
    # Deconvolve
    decoder_layer_3 = deconvolve(layer = decoder_layer_2, k = 4, s = 2, layer_name = "decoderlayer3")

    # Sum (skip connection)
    decoder_layer_4 = tf.add(decoder_layer_3, layer3x, name = "decoderlayer4")
    
    # Deconvolve
    decoderlayer_output = deconvolve(layer = decoder_layer_4, k = 16, s = 8, layer_name = "decoderlayer_output")

    # Return all the layers for a more detailed output
    return vgg_layer_3_out, vgg_layer_4_out, vgg_layer_7_out, layer3x, layer4x, layer7x, \
         decoder_layer_1, decoder_layer_2, decoder_layer_3, decoder_layer_4, decoderlayer_output

In [13]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes = num_classes_):
    """
    TF loss and optimizer operations.
    nn_last_layer: last layer tensor
    correct_label: label image placeholder
    learning_rate: learning rate placeholder
    num_classes: Number of classes to classify
    return: logits, train_op, cross_entropy_loss as python list
    """
    # Flatten 4D tensors to 2D
    # (pixel,class)
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    class_labels = tf.reshape(correct_label, (-1, num_classes))

    # The cross_entropy_loss is the cost heuristic
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = logits,
                                                            labels = class_labels)
    # use the reduce mean method
    cross_entropy_loss = tf.reduce_mean(cross_entropy)

    # Use the standard Adam optimizer to minimize loss
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_loss)

    # return logits, train_op, cross_entropy_loss as python list
    return logits, train_op, cross_entropy_loss

In [14]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op,
             cross_entropy_loss, input_image,
             correct_label, keep_prob_ph, learning_rate):
    """
    Train the neural network and provide debug prints during training
    Arguments: 
    sess: TF Session
    epochs: Number of epochs
    batch_size: Batch size
    get_batches_fn: Function to get batches of training data
    train_op: training operation
    cross_entropy_loss: Loss Tensor
    input_image: TF Placeholder for input images
    correct_label: TF Placeholder for label images
    keep_prob_ph: TF Placeholder for dropout keep probability
    learning_rate: TF Placeholder for learning rate
    """
    # For all epochs
    for epoch in range(epochs):
        #initialize losses and counter
        losses, i = [], 0
        
        # For all images in the batch
        for images, labels in get_batches_fn(batch_size_):
            
            # increment batch counter by 1
            i += 1
            
            # Create the feed by assigining values to placeholders
            feed = {input_image: images,
                    correct_label: labels,
                    keep_prob_ph: dropout_,
                    learning_rate: learning_rate_ }

            # Run the training op with the created feed
            _, partial_loss = sess.run([train_op, cross_entropy_loss], feed_dict = feed)

            # display output
            print("- - - - - >Iteration: ", i, "----->Partial loss:", partial_loss)
            
            # Add to list of losses
            losses.append(partial_loss)

        # After each batch compute net average loss
        training_loss = sum(losses) / len(losses)
        
        # Add to list of global training losses
        all_training_losses.append(training_loss)

        # Print Training loss at end of each Epoch
        print("***************")
        print("Epoch: ", epoch + 1, " of ", epochs_, "training loss: ", training_loss)
        print("***************")


In [15]:
def run_tests():
    """
    Run tests to test whether functions are correctly created
    """
    tests.test_layers(layers)
    tests.test_optimize(optimize)
    tests.test_for_kitti_dataset(data_dir)
    tests.test_train_nn(train_nn)

In [16]:
def run():
    
    print("Training data size", training_size)
    
    # download vgg model if it doesnt exist
    helper.maybe_download_pretrained_vgg(data_dir)
    
    # use the get batches function from the helper.py provided
    get_batches_fn = helper.gen_batch_function(training_dir, img_shape_)
    
    # Using the default session
    with tf.Session() as session:
        
        # Returns the input dropout and output layers from vgg
        image_input, keep_prob_ph, layer_3, layer_4, layer_7 = load_vgg(session, vgg_dir)

        # Create the layers and get the output
        model_output = layers(layer_3, layer_4, layer_7, num_classes_)

        # Get the logits, training op and the loss
        logits, train_op, cross_entropy_loss = optimize(model_output, label_ph, learning_rate_ph, num_classes_)

        # Initilize all variables
        session.run([tf.global_variables_initializer(), tf.local_variables_initializer()])

        # Run the training step
        train_nn(session, epochs_, batch_size_, get_batches_fn, 
                 train_op, cross_entropy_loss, image_input,
                 label_ph, keep_prob_ph, learning_rate_ph)

        # Save inference data
        helper.save_inference_samples(runs_dir, data_dir, session, img_shape_, logits, keep_prob_ph, image_input)

In [17]:
import numpy as np 

def network_shapes():
    
    with tf.Session() as sess:
        # Create a random 3 channel input
        x = np.random.randn(1, 160, 576, 3)

        # Create inputs, dropout and vgg out layers
        image_input, keep_prob_ph, layer_3, layer_4, layer_7 = load_vgg(sess, vgg_dir)
        
        # Create verbose layers
        op = layers_verbose(layer_3, layer_4, layer_7, num_classes_)

        # initialize the variables 
        sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])

        # Run the optimizer
        l3, l4, l7, l3x, l4x, l7x, d1, s2, d3, s4, d5 = sess.run(op, feed_dict = {image_input: x, keep_prob_ph: 1.0})

        print("------------------")
        print("shapes of layers:") 
        print("------------------")

        print("layer3 -->", l3.shape)
        print("layer4 -->", l4.shape)
        print("layer7 -->", l7.shape)
        print("layer3 conv1x1 -->", l3x.shape)
        print("layer4 conv1x1 -->", l4x.shape)
        print("layer7 conv1x1-->", l7x.shape)
        print("decoderlayer1 transpose: layer7 k = 4 s = 2 -->", d1.shape)
        print("decoderlayer2 skip: decoderlayer1 and layer4conv1x1 -->", s2.shape)
        print("decoderlayer3 transpose: decoderlayer2 k = 4 s = 2 -->", d3.shape)
        print("decoderlayer4 skip: decoderlayer3 and layer3conv1x1 -->", s4.shape)
        print("decoderlayer5 transpose: decoderlayer4 k = 16 s = 8 -->", d5.shape)

In [18]:
run()

Training data size 289
INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
- - - - - >Iteration:  1 ----->Partial loss: 73.7186
- - - - - >Iteration:  2 ----->Partial loss: 57.7373
- - - - - >Iteration:  3 ----->Partial loss: 37.2718
- - - - - >Iteration:  4 ----->Partial loss: 37.1428
- - - - - >Iteration:  5 ----->Partial loss: 25.4312
- - - - - >Iteration:  6 ----->Partial loss: 20.9919
- - - - - >Iteration:  7 ----->Partial loss: 19.5452
- - - - - >Iteration:  8 ----->Partial loss: 20.5204
- - - - - >Iteration:  9 ----->Partial loss: 19.028
- - - - - >Iteration:  10 ----->Partial loss: 12.4205
- - - - - >Iteration:  11 ----->Partial loss: 12.2238
- - - - - >Iteration:  12 ----->Partial loss: 11.8303
- - - - - >Iteration:  13 ----->Partial loss: 8.89572
- - - - - >Iteration:  14 ----->Partial loss: 8.93831
- - - - - >Iteration:  15 ----->Partial loss: 9.00388
- - - - - >Iteration:  16 ----->Partial loss: 8.04923
- - - - - >Iteration:  17 ----->Partial loss: 6

- - - - - >Iteration:  150 ----->Partial loss: 0.845993
- - - - - >Iteration:  151 ----->Partial loss: 0.81113
- - - - - >Iteration:  152 ----->Partial loss: 0.807778
- - - - - >Iteration:  153 ----->Partial loss: 0.851702
- - - - - >Iteration:  154 ----->Partial loss: 0.867429
- - - - - >Iteration:  155 ----->Partial loss: 0.800666
- - - - - >Iteration:  156 ----->Partial loss: 0.809482
- - - - - >Iteration:  157 ----->Partial loss: 0.806437
- - - - - >Iteration:  158 ----->Partial loss: 0.807842
- - - - - >Iteration:  159 ----->Partial loss: 0.802028
- - - - - >Iteration:  160 ----->Partial loss: 0.795434
- - - - - >Iteration:  161 ----->Partial loss: 0.882614
- - - - - >Iteration:  162 ----->Partial loss: 0.766775
- - - - - >Iteration:  163 ----->Partial loss: 0.781632
- - - - - >Iteration:  164 ----->Partial loss: 0.861302
- - - - - >Iteration:  165 ----->Partial loss: 0.789031
- - - - - >Iteration:  166 ----->Partial loss: 0.822652
- - - - - >Iteration:  167 ----->Partial loss: 0.

- - - - - >Iteration:  7 ----->Partial loss: 0.700492
- - - - - >Iteration:  8 ----->Partial loss: 0.727271
- - - - - >Iteration:  9 ----->Partial loss: 0.734071
- - - - - >Iteration:  10 ----->Partial loss: 0.724164
- - - - - >Iteration:  11 ----->Partial loss: 0.729724
- - - - - >Iteration:  12 ----->Partial loss: 0.707706
- - - - - >Iteration:  13 ----->Partial loss: 0.715585
- - - - - >Iteration:  14 ----->Partial loss: 0.707698
- - - - - >Iteration:  15 ----->Partial loss: 0.724249
- - - - - >Iteration:  16 ----->Partial loss: 0.696909
- - - - - >Iteration:  17 ----->Partial loss: 0.718448
- - - - - >Iteration:  18 ----->Partial loss: 0.697315
- - - - - >Iteration:  19 ----->Partial loss: 0.699252
- - - - - >Iteration:  20 ----->Partial loss: 0.697195
- - - - - >Iteration:  21 ----->Partial loss: 0.707544
- - - - - >Iteration:  22 ----->Partial loss: 0.73802
- - - - - >Iteration:  23 ----->Partial loss: 0.720471
- - - - - >Iteration:  24 ----->Partial loss: 0.741751
- - - - - >Ite

- - - - - >Iteration:  156 ----->Partial loss: 0.674355
- - - - - >Iteration:  157 ----->Partial loss: 0.682181
- - - - - >Iteration:  158 ----->Partial loss: 0.672396
- - - - - >Iteration:  159 ----->Partial loss: 0.680557
- - - - - >Iteration:  160 ----->Partial loss: 0.682564
- - - - - >Iteration:  161 ----->Partial loss: 0.698767
- - - - - >Iteration:  162 ----->Partial loss: 0.691726
- - - - - >Iteration:  163 ----->Partial loss: 0.673583
- - - - - >Iteration:  164 ----->Partial loss: 0.677832
- - - - - >Iteration:  165 ----->Partial loss: 0.683648
- - - - - >Iteration:  166 ----->Partial loss: 0.674258
- - - - - >Iteration:  167 ----->Partial loss: 0.664041
- - - - - >Iteration:  168 ----->Partial loss: 0.690572
- - - - - >Iteration:  169 ----->Partial loss: 0.668832
- - - - - >Iteration:  170 ----->Partial loss: 0.670415
- - - - - >Iteration:  171 ----->Partial loss: 0.686139
- - - - - >Iteration:  172 ----->Partial loss: 0.677809
- - - - - >Iteration:  173 ----->Partial loss: 0

- - - - - >Iteration:  13 ----->Partial loss: 0.658896
- - - - - >Iteration:  14 ----->Partial loss: 0.659216
- - - - - >Iteration:  15 ----->Partial loss: 0.648426
- - - - - >Iteration:  16 ----->Partial loss: 0.646717
- - - - - >Iteration:  17 ----->Partial loss: 0.658481
- - - - - >Iteration:  18 ----->Partial loss: 0.665759
- - - - - >Iteration:  19 ----->Partial loss: 0.654323
- - - - - >Iteration:  20 ----->Partial loss: 0.651052
- - - - - >Iteration:  21 ----->Partial loss: 0.657405
- - - - - >Iteration:  22 ----->Partial loss: 0.64796
- - - - - >Iteration:  23 ----->Partial loss: 0.646855
- - - - - >Iteration:  24 ----->Partial loss: 0.654116
- - - - - >Iteration:  25 ----->Partial loss: 0.662102
- - - - - >Iteration:  26 ----->Partial loss: 0.647327
- - - - - >Iteration:  27 ----->Partial loss: 0.658146
- - - - - >Iteration:  28 ----->Partial loss: 0.646403
- - - - - >Iteration:  29 ----->Partial loss: 0.660627
- - - - - >Iteration:  30 ----->Partial loss: 0.655001
- - - - - >

...

- - - - - >Iteration:  203 ----->Partial loss: 0.0427529
- - - - - >Iteration:  204 ----->Partial loss: 0.0523681
- - - - - >Iteration:  205 ----->Partial loss: 0.0611443
- - - - - >Iteration:  206 ----->Partial loss: 0.0407611
- - - - - >Iteration:  207 ----->Partial loss: 0.0271534
- - - - - >Iteration:  208 ----->Partial loss: 0.0842178
- - - - - >Iteration:  209 ----->Partial loss: 0.0201427
- - - - - >Iteration:  210 ----->Partial loss: 0.0615365
- - - - - >Iteration:  211 ----->Partial loss: 0.0276625
- - - - - >Iteration:  212 ----->Partial loss: 0.0578162
- - - - - >Iteration:  213 ----->Partial loss: 0.0375062
- - - - - >Iteration:  214 ----->Partial loss: 0.0558792
- - - - - >Iteration:  215 ----->Partial loss: 0.0326467
- - - - - >Iteration:  216 ----->Partial loss: 0.0657881
- - - - - >Iteration:  217 ----->Partial loss: 0.0341761
- - - - - >Iteration:  218 ----->Partial loss: 0.0583102
- - - - - >Iteration:  219 ----->Partial loss: 0.0497262
- - - - - >Iteration:  220 ----

- - - - - >Iteration:  58 ----->Partial loss: 0.0384835
- - - - - >Iteration:  59 ----->Partial loss: 0.0569299
- - - - - >Iteration:  60 ----->Partial loss: 0.0729364
- - - - - >Iteration:  61 ----->Partial loss: 0.0176293
- - - - - >Iteration:  62 ----->Partial loss: 0.0234087
- - - - - >Iteration:  63 ----->Partial loss: 0.0835781
- - - - - >Iteration:  64 ----->Partial loss: 0.0564459
- - - - - >Iteration:  65 ----->Partial loss: 0.0695205
- - - - - >Iteration:  66 ----->Partial loss: 0.0329145
- - - - - >Iteration:  67 ----->Partial loss: 0.0638219
- - - - - >Iteration:  68 ----->Partial loss: 0.0174569
- - - - - >Iteration:  69 ----->Partial loss: 0.0240342
- - - - - >Iteration:  70 ----->Partial loss: 0.0433384
- - - - - >Iteration:  71 ----->Partial loss: 0.0468117
- - - - - >Iteration:  72 ----->Partial loss: 0.0337167
- - - - - >Iteration:  73 ----->Partial loss: 0.025207
- - - - - >Iteration:  74 ----->Partial loss: 0.0636339
- - - - - >Iteration:  75 ----->Partial loss: 0.0

- - - - - >Iteration:  203 ----->Partial loss: 0.0629172
- - - - - >Iteration:  204 ----->Partial loss: 0.0633882
- - - - - >Iteration:  205 ----->Partial loss: 0.0296088
- - - - - >Iteration:  206 ----->Partial loss: 0.0698123
- - - - - >Iteration:  207 ----->Partial loss: 0.0247773
- - - - - >Iteration:  208 ----->Partial loss: 0.0415263
- - - - - >Iteration:  209 ----->Partial loss: 0.0423466
- - - - - >Iteration:  210 ----->Partial loss: 0.0523625
- - - - - >Iteration:  211 ----->Partial loss: 0.0176036
- - - - - >Iteration:  212 ----->Partial loss: 0.0656363
- - - - - >Iteration:  213 ----->Partial loss: 0.0554245
- - - - - >Iteration:  214 ----->Partial loss: 0.0680479
- - - - - >Iteration:  215 ----->Partial loss: 0.0307445
- - - - - >Iteration:  216 ----->Partial loss: 0.0580732
- - - - - >Iteration:  217 ----->Partial loss: 0.0185473
- - - - - >Iteration:  218 ----->Partial loss: 0.0512995
- - - - - >Iteration:  219 ----->Partial loss: 0.0353344
- - - - - >Iteration:  220 ----

- - - - - >Iteration:  58 ----->Partial loss: 0.0149648
- - - - - >Iteration:  59 ----->Partial loss: 0.0638513
- - - - - >Iteration:  60 ----->Partial loss: 0.042743
- - - - - >Iteration:  61 ----->Partial loss: 0.0412068
- - - - - >Iteration:  62 ----->Partial loss: 0.0476743
- - - - - >Iteration:  63 ----->Partial loss: 0.0480848
- - - - - >Iteration:  64 ----->Partial loss: 0.0244154
- - - - - >Iteration:  65 ----->Partial loss: 0.0173098
- - - - - >Iteration:  66 ----->Partial loss: 0.0572133
- - - - - >Iteration:  67 ----->Partial loss: 0.0247849
- - - - - >Iteration:  68 ----->Partial loss: 0.0366062
- - - - - >Iteration:  69 ----->Partial loss: 0.0307824
- - - - - >Iteration:  70 ----->Partial loss: 0.0427546
- - - - - >Iteration:  71 ----->Partial loss: 0.0539688
- - - - - >Iteration:  72 ----->Partial loss: 0.0971707
- - - - - >Iteration:  73 ----->Partial loss: 0.0290373
- - - - - >Iteration:  74 ----->Partial loss: 0.0263919
- - - - - >Iteration:  75 ----->Partial loss: 0.0

- - - - - >Iteration:  203 ----->Partial loss: 0.0508945
- - - - - >Iteration:  204 ----->Partial loss: 0.0316139
- - - - - >Iteration:  205 ----->Partial loss: 0.0428977
- - - - - >Iteration:  206 ----->Partial loss: 0.0753169
- - - - - >Iteration:  207 ----->Partial loss: 0.0605241
- - - - - >Iteration:  208 ----->Partial loss: 0.0431367
- - - - - >Iteration:  209 ----->Partial loss: 0.064796
- - - - - >Iteration:  210 ----->Partial loss: 0.053781
- - - - - >Iteration:  211 ----->Partial loss: 0.0463295
- - - - - >Iteration:  212 ----->Partial loss: 0.0537562
- - - - - >Iteration:  213 ----->Partial loss: 0.0289772
- - - - - >Iteration:  214 ----->Partial loss: 0.0752088
- - - - - >Iteration:  215 ----->Partial loss: 0.0411044
- - - - - >Iteration:  216 ----->Partial loss: 0.0308116
- - - - - >Iteration:  217 ----->Partial loss: 0.0191307
- - - - - >Iteration:  218 ----->Partial loss: 0.0509622
- - - - - >Iteration:  219 ----->Partial loss: 0.067949
- - - - - >Iteration:  220 ----->P

In [19]:
network_shapes()

INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'


ValueError: Variable layer3conv1x1/kernel already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-9-684fcb842599>", line 7, in conv_1x1
    name = layer_name)
  File "<ipython-input-11-97e822571b92>", line 11, in layers
    layer_3x = conv_1x1(layer = vgg_layer_3_out, layer_name = "layer3conv1x1")
  File "<ipython-input-16-1d5ce9f4975f>", line 18, in run
    model_output = layers(layer_3, layer_4, layer_7, num_classes_)
